# 토큰 사용량 확인

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# Bedrock 설정 및 Util 함수 설정
from langchain_aws import ChatBedrockConverse
import os
from Util.stream_utils import print_stream_content, get_stream_content

# 객체 생성
llm = ChatBedrockConverse(
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    model="apac.anthropic.claude-sonnet-4-20250514-v1:0",
    region_name="ap-northeast-2",
)

In [3]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH04-Models")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH04-Models


특정 호출에 대한 토큰 사용량을 추적하는 방법에 대해 설명합니다.

이 기능은 현재 OpenAI API 에만 구현되어 있습니다.

먼저 단일 Chat 모델 호출에 대한 토큰 사용량을 추적하는 매우 간단한 예를 살펴보겠습니다.


In [4]:
# Bedrock에서 토큰 사용량 추적 예시
message = "대한민국의 수도는 어디야?"

# 모델 호출
response = llm.invoke(message)

print("=== Bedrock 토큰 사용량 정보 ===")
print(f"응답 내용: {response.content}")
print()

# usage_metadata에서 토큰 정보 확인
if hasattr(response, "usage_metadata") and response.usage_metadata:
    usage_info = response.usage_metadata
    print(f"입력 토큰 수: {usage_info.get('input_tokens', 'N/A')}")
    print(f"출력 토큰 수: {usage_info.get('output_tokens', 'N/A')}")
    print(f"총 토큰 수: {usage_info.get('total_tokens', 'N/A')}")
else:
    print("토큰 사용량 정보를 사용할 수 없습니다.")

print()

# response_metadata에서 추가 정보 확인
if hasattr(response, "response_metadata") and response.response_metadata:
    print("=== 응답 메타데이터 ===")
    metadata = response.response_metadata
    print(f"Stop Reason: {metadata.get('stopReason', 'N/A')}")

    # 메트릭 정보 (지연시간 등)
    if "metrics" in metadata:
        metrics = metadata["metrics"]
        print(f"지연시간: {metrics.get('latencyMs', 'N/A')} ms")

=== Bedrock 토큰 사용량 정보 ===
응답 내용: 대한민국의 수도는 서울입니다.

입력 토큰 수: 22
출력 토큰 수: 20
총 토큰 수: 42

=== 응답 메타데이터 ===
Stop Reason: end_turn
지연시간: [1013] ms


In [ ]:
from langchain.callbacks import get_openai_callback
from langchain_openai import ChatOpenAI

# 모델을 불러옵니다.
llm = ChatOpenAI(model_name="gpt-4.1-mini")

`with get_openai_callback()` 구문안에서 실행되는 모든 토큰 사용량/요금이 추적됩니다.

In [ ]:
# callback을 사용하여 추적합니다.
with get_openai_callback() as cb:
    result = llm.invoke("대한민국의 수도는 어디야?")
    print(cb)

In [ ]:
# callback을 사용하여 추적합니다.
with get_openai_callback() as cb:
    result = llm.invoke("대한민국의 수도는 어디야?")
    result = llm.invoke("대한민국의 수도는 어디야?")
    print(f"총 사용된 토큰수: \t\t{cb.total_tokens}")
    print(f"프롬프트에 사용된 토큰수: \t{cb.prompt_tokens}")
    print(f"답변에 사용된 토큰수: \t{cb.completion_tokens}")
    print(f"호출에 청구된 금액(USD): \t${cb.total_cost}")